In [ ]:
import numpy as np
import os, sys
import dm_utils as ut
sys.path.append('../')  # the code for fair classification is in this directory

import loaders.load_compas_data as compas
import loaders.load_bank as bank
import loaders.load_adult as adult
from dis_meas_class import DispMistreatmentClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import seaborn as sns

In [ ]:
def dFNR(y_true, y_pred, X, sa_index, sa_label):
    sa_pos = (X[:, sa_index] == sa_label) * (y_true == 1)
    sa_neg = (X[:, sa_index] == sa_label) * (y_true == -1)
    nonsa_pos = (X[:, sa_index] != sa_label) * (y_true == 1)
    nonsa_neg = (X[:, sa_index] != sa_label) * (y_true == -1)
    return np.sum(y_pred[sa_pos] != y_true[sa_pos]) / np.sum(sa_pos) - np.sum(y_pred[nonsa_pos] != y_true[nonsa_pos]) / np.sum(nonsa_pos)
def dFPR(y_true, y_pred, X, sa_index, sa_label):
    sa_pos = (X[:, sa_index] == sa_label) * (y_true == 1)
    sa_neg = (X[:, sa_index] == sa_label) * (y_true == -1)
    nonsa_pos = (X[:, sa_index] != sa_label) * (y_true == 1)
    nonsa_neg = (X[:, sa_index] != sa_label) * (y_true == -1)

    return np.sum(y_pred[sa_neg] != y_true[sa_neg]) / np.sum(sa_neg) - np.sum(y_pred[nonsa_neg] != y_true[nonsa_neg]) / np.sum(nonsa_neg)
def TPR(y_true, y_pred, X, sa_index, sa_label, agg):
    sa_pos = (X[:, sa_index] == sa_label) * (y_true == 1)
    sa_neg = (X[:, sa_index] == sa_label) * (y_true == -1)
    nonsa_pos = (X[:, sa_index] != sa_label) * (y_true == 1)
    nonsa_neg = (X[:, sa_index] != sa_label) * (y_true == -1)

    if agg == 'prot':
        return np.sum(y_pred[nonsa_pos] == y_true[nonsa_pos]) / np.sum(nonsa_pos)
    elif agg == 'non-prot':
        return np.sum(y_pred[sa_pos] == y_true[sa_pos]) / np.sum(sa_pos)
def TNR(y_true, y_pred, X, sa_index, sa_label, agg='diff'):
    sa_pos = (X[:, sa_index] == sa_label) * (y_true == 1)
    sa_neg = (X[:, sa_index] == sa_label) * (y_true == -1)
    nonsa_pos = (X[:, sa_index] != sa_label) * (y_true == 1)
    nonsa_neg = (X[:, sa_index] != sa_label) * (y_true == -1)

    if agg == 'prot':
        return np.sum(y_pred[nonsa_neg] == y_true[nonsa_neg]) / np.sum(nonsa_neg)
    elif agg == 'non-prot':
        return np.sum(y_pred[sa_neg] == y_true[sa_neg]) / np.sum(sa_neg)
def model_evaluate(y_true, y_pred, X, sa_index, sa_label):
    metrics = {'Accuracy': accuracy_score(y_true, y_pred), 'Bal. Acc.': balanced_accuracy_score(y_true, y_pred),
              'Eq.Odds': abs(dFPR(y_true, y_pred, X, sa_index, sa_label)) + abs(dFNR(y_true, y_pred, X, sa_index, sa_label)),
              'TPR Prot': TPR(y_true, y_pred, X, sa_index, sa_label, agg='prot'),
              'TPR Non-Prot': TPR(y_true, y_pred, X, sa_index, sa_label, agg='non-prot'),
              'TNR Prot': TNR(y_true, y_pred, X, sa_index, sa_label, agg='prot'),
              'TNR Non-Prot': TNR(y_true, y_pred, X, sa_index, sa_label, agg='non-prot')}
    return metrics

In [ ]:
train_fold_size = 0.5
constr = 1
X, y, sa_index, _, x_control = compas.load_compas()
sa_label=1
nonprot = X[:, sa_index]==sa_label
prot = X[:, sa_index]!=sa_label

names = ['Accuracy', 'Bal. Acc.', 'Eq.Odds', 'TPR Prot', 'TPR Non-Prot', 'TNR Prot', 'TNR Non-Prot']
metrics = {'name':[], 'value':[], 'model':[]}
for i in range(10):
    x_train, y_train, \
    x_control_train, \
    x_test, y_test, \
    x_control_test = ut.split_into_train_test(X, y, x_control,train_fold_size)

    clf1 = DispMistreatmentClassifier(x_control_train,cons_type=constr)
    clf1.fit(x_train, y_train)
    y_pred_dm = clf1.predict(x_test)

    metrics['name'].append(names[0])
    metrics['value'].append(accuracy_score(y_test, y_pred_dm))
    metrics['model'].append('DispMist_FPR')
    metrics['name'].append(names[1])
    metrics['value'].append(balanced_accuracy_score(y_test, y_pred_dm))
    metrics['model'].append('DispMist_FPR')
    metrics['name'].append(names[2])
    metrics['value'].append(abs(dFPR(y_test, y_pred_dm, x_test, sa_index, sa_label)) + abs(dFNR(y_test, y_pred_dm, x_test, sa_index, sa_label)))
    metrics['model'].append('DispMist_FPR')
    metrics['name'].append(names[3])
    metrics['value'].append(TPR(y_test, y_pred_dm, x_test, sa_index, sa_label, agg='prot'))
    metrics['model'].append('DispMist_FPR')
    metrics['name'].append(names[4])
    metrics['value'].append(TPR(y_test, y_pred_dm, x_test, sa_index, sa_label, agg='non-prot'))
    metrics['model'].append('DispMist_FPR')
    metrics['name'].append(names[5])
    metrics['value'].append(TNR(y_test, y_pred_dm, x_test, sa_index, sa_label, agg='prot'))
    metrics['model'].append('DispMist_FPR')
    metrics['name'].append(names[6])
    metrics['value'].append(TNR(y_test, y_pred_dm, x_test, sa_index, sa_label, agg='non-prot'))
    metrics['model'].append('DispMist_FPR')

    clf2 = RandomForestClassifier(n_estimators=100, random_state=0xB00B1E5)
    clf2.fit(x_train, y_train)
    y_pred_rf = clf2.predict(x_test)

    metrics['name'].append(names[0])
    metrics['value'].append(accuracy_score(y_test, y_pred_rf))
    metrics['model'].append('RandomForest')
    metrics['name'].append(names[1])
    metrics['value'].append(balanced_accuracy_score(y_test, y_pred_rf))
    metrics['model'].append('RandomForest')
    metrics['name'].append(names[2])
    metrics['value'].append(abs(dFPR(y_test, y_pred_rf, x_test, sa_index, sa_label)) + abs(dFNR(y_test, y_pred_rf, x_test, sa_index, sa_label)))
    metrics['model'].append('RandomForest')
    metrics['name'].append(names[3])
    metrics['value'].append(TPR(y_test, y_pred_rf, x_test, sa_index, sa_label, agg='prot'))
    metrics['model'].append('RandomForest')
    metrics['name'].append(names[4])
    metrics['value'].append(TPR(y_test, y_pred_rf, x_test, sa_index, sa_label, agg='non-prot'))
    metrics['model'].append('RandomForest')
    metrics['name'].append(names[5])
    metrics['value'].append(TNR(y_test, y_pred_rf, x_test, sa_index, sa_label, agg='prot'))
    metrics['model'].append('RandomForest')
    metrics['name'].append(names[6])
    metrics['value'].append(TNR(y_test, y_pred_rf, x_test, sa_index, sa_label, agg='non-prot'))
    metrics['model'].append('RandomForest')

    clf3 = LogisticRegression(random_state=0xB00B1E5)
    clf3.fit(x_train, y_train)

    y_pred_lr = clf3.predict(x_test)
    metrics['name'].append(names[0])
    metrics['value'].append(accuracy_score(y_test, y_pred_lr))
    metrics['model'].append('LogisticRegression')
    metrics['name'].append(names[1])
    metrics['value'].append(balanced_accuracy_score(y_test, y_pred_lr))
    metrics['model'].append('LogisticRegression')
    metrics['name'].append(names[2])
    metrics['value'].append(abs(dFPR(y_test, y_pred_lr, x_test, sa_index, sa_label)) + abs(dFNR(y_test, y_pred_lr, x_test, sa_index, sa_label)))
    metrics['model'].append('LogisticRegression')
    metrics['name'].append(names[3])
    metrics['value'].append(TPR(y_test, y_pred_lr, x_test, sa_index, sa_label, agg='prot'))
    metrics['model'].append('LogisticRegression')
    metrics['name'].append(names[4])
    metrics['value'].append(TPR(y_test, y_pred_lr, x_test, sa_index, sa_label, agg='non-prot'))
    metrics['model'].append('LogisticRegression')
    metrics['name'].append(names[5])
    metrics['value'].append(TNR(y_test, y_pred_lr, x_test, sa_index, sa_label, agg='prot'))
    metrics['model'].append('LogisticRegression')
    metrics['name'].append(names[6])
    metrics['value'].append(TNR(y_test, y_pred_lr, x_test, sa_index, sa_label, agg='non-prot'))
    metrics['model'].append('LogisticRegression')

Optimization done, problem status: Converged
Optimization done, problem status: Converged
Optimization done, problem status: Converged
Optimization done, problem status: Converged
Optimization done, problem status: Converged
Optimization done, problem status: Converged
Optimization done, problem status: Converged
Optimization done, problem status: Converged
Optimization done, problem status: Converged


In [ ]:
f = open('metrics_compass.txt','w')
f.write(str(metrics))
f.close()

In [15]:
train_fold_size = 0.4
X, y, sa_index, _, x_control = adult.load_adult("sex")
sa_label = 0
nonprot = X[:, sa_index]==sa_label
prot = X[:, sa_index]!=sa_label

names = ['Accuracy', 'Bal. Acc.', 'Eq.Odds', 'TPR Prot', 'TPR Non-Prot', 'TNR Prot', 'TNR Non-Prot']
metrics = {'name':[], 'value':[], 'model':[]}
for i in range(10):
    x_train, y_train, \
    x_control_train, \
    x_test, y_test, \
    x_control_test = ut.split_into_train_test(X, y, x_control,train_fold_size)

    clf1 = DispMistreatmentClassifier(x_control_train,cons_type=constr)
    clf1.fit(x_train, y_train)
    y_pred_dm = clf1.predict(x_test)

    metrics['name'].append(names[0])
    metrics['value'].append(accuracy_score(y_test, y_pred_dm))
    metrics['model'].append('DispMist_FPR')
    metrics['name'].append(names[1])
    metrics['value'].append(balanced_accuracy_score(y_test, y_pred_dm))
    metrics['model'].append('DispMist_FPR')
    metrics['name'].append(names[2])
    metrics['value'].append(abs(dFPR(y_test, y_pred_dm, x_test, sa_index, sa_label)) + abs(dFNR(y_test, y_pred_dm, x_test, sa_index, sa_label)))
    metrics['model'].append('DispMist_FPR')
    metrics['name'].append(names[3])
    metrics['value'].append(TPR(y_test, y_pred_dm, x_test, sa_index, sa_label, agg='prot'))
    metrics['model'].append('DispMist_FPR')
    metrics['name'].append(names[4])
    metrics['value'].append(TPR(y_test, y_pred_dm, x_test, sa_index, sa_label, agg='non-prot'))
    metrics['model'].append('DispMist_FPR')
    metrics['name'].append(names[5])
    metrics['value'].append(TNR(y_test, y_pred_dm, x_test, sa_index, sa_label, agg='prot'))
    metrics['model'].append('DispMist_FPR')
    metrics['name'].append(names[6])
    metrics['value'].append(TNR(y_test, y_pred_dm, x_test, sa_index, sa_label, agg='non-prot'))
    metrics['model'].append('DispMist_FPR')

    clf2 = RandomForestClassifier(n_estimators=100, random_state=0xB00B1E5)
    clf2.fit(x_train, y_train)
    y_pred_rf = clf2.predict(x_test)

    metrics['name'].append(names[0])
    metrics['value'].append(accuracy_score(y_test, y_pred_rf))
    metrics['model'].append('RandomForest')
    metrics['name'].append(names[1])
    metrics['value'].append(balanced_accuracy_score(y_test, y_pred_rf))
    metrics['model'].append('RandomForest')
    metrics['name'].append(names[2])
    metrics['value'].append(abs(dFPR(y_test, y_pred_rf, x_test, sa_index, sa_label)) + abs(dFNR(y_test, y_pred_rf, x_test, sa_index, sa_label)))
    metrics['model'].append('RandomForest')
    metrics['name'].append(names[3])
    metrics['value'].append(TPR(y_test, y_pred_rf, x_test, sa_index, sa_label, agg='prot'))
    metrics['model'].append('RandomForest')
    metrics['name'].append(names[4])
    metrics['value'].append(TPR(y_test, y_pred_rf, x_test, sa_index, sa_label, agg='non-prot'))
    metrics['model'].append('RandomForest')
    metrics['name'].append(names[5])
    metrics['value'].append(TNR(y_test, y_pred_rf, x_test, sa_index, sa_label, agg='prot'))
    metrics['model'].append('RandomForest')
    metrics['name'].append(names[6])
    metrics['value'].append(TNR(y_test, y_pred_rf, x_test, sa_index, sa_label, agg='non-prot'))
    metrics['model'].append('RandomForest')

    clf3 = LogisticRegression(random_state=0xB00B1E5,max_iter=1000)
    clf3.fit(x_train, y_train)

    y_pred_lr = clf3.predict(x_test)
    metrics['name'].append(names[0])
    metrics['value'].append(accuracy_score(y_test, y_pred_lr))
    metrics['model'].append('LogisticRegression')
    metrics['name'].append(names[1])
    metrics['value'].append(balanced_accuracy_score(y_test, y_pred_lr))
    metrics['model'].append('LogisticRegression')
    metrics['name'].append(names[2])
    metrics['value'].append(abs(dFPR(y_test, y_pred_lr, x_test, sa_index, sa_label)) + abs(dFNR(y_test, y_pred_lr, x_test, sa_index, sa_label)))
    metrics['model'].append('LogisticRegression')
    metrics['name'].append(names[3])
    metrics['value'].append(TPR(y_test, y_pred_lr, x_test, sa_index, sa_label, agg='prot'))
    metrics['model'].append('LogisticRegression')
    metrics['name'].append(names[4])
    metrics['value'].append(TPR(y_test, y_pred_lr, x_test, sa_index, sa_label, agg='non-prot'))
    metrics['model'].append('LogisticRegression')
    metrics['name'].append(names[5])
    metrics['value'].append(TNR(y_test, y_pred_lr, x_test, sa_index, sa_label, agg='prot'))
    metrics['model'].append('LogisticRegression')
    metrics['name'].append(names[6])
    metrics['value'].append(TNR(y_test, y_pred_lr, x_test, sa_index, sa_label, agg='non-prot'))
    metrics['model'].append('LogisticRegression')

Optimization done, problem status: Converged
Optimization done, problem status: Converged
Optimization done, problem status: Converged
Optimization done, problem status: Converged
Optimization done, problem status: Converged
Optimization done, problem status: Converged
Optimization done, problem status: Converged
Optimization done, problem status: Converged
Optimization done, problem status: Converged
Optimization done, problem status: Converged


In [16]:
f = open('metrics_adult.txt','w')
f.write(str(metrics))
f.close()

In [13]:
X, y, sa_index, p_Group, x_control = bank.load_bank()
sa_label=1
nonprot = X[:, sa_index]==sa_label
prot = X[:, sa_index]!=sa_label
train_fold_size = 0.5

names = ['Accuracy', 'Bal. Acc.', 'Eq.Odds', 'TPR Prot', 'TPR Non-Prot', 'TNR Prot', 'TNR Non-Prot']
metrics = {'name':[], 'value':[], 'model':[]}
for i in range(10):
    x_train, y_train, \
    x_control_train, \
    x_test, y_test, \
    x_control_test = ut.split_into_train_test(X, y, x_control,train_fold_size)

    clf1 = DispMistreatmentClassifier(x_control_train,cons_type=constr)
    clf1.fit(x_train, y_train)
    y_pred_dm = clf1.predict(x_test)

    metrics['name'].append(names[0])
    metrics['value'].append(accuracy_score(y_test, y_pred_dm))
    metrics['model'].append('DispMist_FPR')
    metrics['name'].append(names[1])
    metrics['value'].append(balanced_accuracy_score(y_test, y_pred_dm))
    metrics['model'].append('DispMist_FPR')
    metrics['name'].append(names[2])
    metrics['value'].append(abs(dFPR(y_test, y_pred_dm, x_test, sa_index, sa_label)) + abs(dFNR(y_test, y_pred_dm, x_test, sa_index, sa_label)))
    metrics['model'].append('DispMist_FPR')
    metrics['name'].append(names[3])
    metrics['value'].append(TPR(y_test, y_pred_dm, x_test, sa_index, sa_label, agg='prot'))
    metrics['model'].append('DispMist_FPR')
    metrics['name'].append(names[4])
    metrics['value'].append(TPR(y_test, y_pred_dm, x_test, sa_index, sa_label, agg='non-prot'))
    metrics['model'].append('DispMist_FPR')
    metrics['name'].append(names[5])
    metrics['value'].append(TNR(y_test, y_pred_dm, x_test, sa_index, sa_label, agg='prot'))
    metrics['model'].append('DispMist_FPR')
    metrics['name'].append(names[6])
    metrics['value'].append(TNR(y_test, y_pred_dm, x_test, sa_index, sa_label, agg='non-prot'))
    metrics['model'].append('DispMist_FPR')

    clf2 = RandomForestClassifier(n_estimators=100, random_state=0xB00B1E5)
    clf2.fit(x_train, y_train)
    y_pred_rf = clf2.predict(x_test)

    metrics['name'].append(names[0])
    metrics['value'].append(accuracy_score(y_test, y_pred_rf))
    metrics['model'].append('RandomForest')
    metrics['name'].append(names[1])
    metrics['value'].append(balanced_accuracy_score(y_test, y_pred_rf))
    metrics['model'].append('RandomForest')
    metrics['name'].append(names[2])
    metrics['value'].append(abs(dFPR(y_test, y_pred_rf, x_test, sa_index, sa_label)) + abs(dFNR(y_test, y_pred_rf, x_test, sa_index, sa_label)))
    metrics['model'].append('RandomForest')
    metrics['name'].append(names[3])
    metrics['value'].append(TPR(y_test, y_pred_rf, x_test, sa_index, sa_label, agg='prot'))
    metrics['model'].append('RandomForest')
    metrics['name'].append(names[4])
    metrics['value'].append(TPR(y_test, y_pred_rf, x_test, sa_index, sa_label, agg='non-prot'))
    metrics['model'].append('RandomForest')
    metrics['name'].append(names[5])
    metrics['value'].append(TNR(y_test, y_pred_rf, x_test, sa_index, sa_label, agg='prot'))
    metrics['model'].append('RandomForest')
    metrics['name'].append(names[6])
    metrics['value'].append(TNR(y_test, y_pred_rf, x_test, sa_index, sa_label, agg='non-prot'))
    metrics['model'].append('RandomForest')

    clf3 = LogisticRegression(random_state=0xB00B1E5)
    clf3.fit(x_train, y_train)

    y_pred_lr = clf3.predict(x_test)
    metrics['name'].append(names[0])
    metrics['value'].append(accuracy_score(y_test, y_pred_lr))
    metrics['model'].append('LogisticRegression')
    metrics['name'].append(names[1])
    metrics['value'].append(balanced_accuracy_score(y_test, y_pred_lr))
    metrics['model'].append('LogisticRegression')
    metrics['name'].append(names[2])
    metrics['value'].append(abs(dFPR(y_test, y_pred_lr, x_test, sa_index, sa_label)) + abs(dFNR(y_test, y_pred_lr, x_test, sa_index, sa_label)))
    metrics['model'].append('LogisticRegression')
    metrics['name'].append(names[3])
    metrics['value'].append(TPR(y_test, y_pred_lr, x_test, sa_index, sa_label, agg='prot'))
    metrics['model'].append('LogisticRegression')
    metrics['name'].append(names[4])
    metrics['value'].append(TPR(y_test, y_pred_lr, x_test, sa_index, sa_label, agg='non-prot'))
    metrics['model'].append('LogisticRegression')
    metrics['name'].append(names[5])
    metrics['value'].append(TNR(y_test, y_pred_lr, x_test, sa_index, sa_label, agg='prot'))
    metrics['model'].append('LogisticRegression')
    metrics['name'].append(names[6])
    metrics['value'].append(TNR(y_test, y_pred_lr, x_test, sa_index, sa_label, agg='non-prot'))
    metrics['model'].append('LogisticRegression')

Optimization done, problem status: Converged
Optimization done, problem status: Converged
Optimization done, problem status: Converged
Optimization done, problem status: Converged
Optimization done, problem status: Converged
Optimization done, problem status: Converged
Optimization done, problem status: Converged
Optimization done, problem status: Converged
Optimization done, problem status: Converged
Optimization done, problem status: Converged


In [14]:
f = open('metrics_bank.txt','w')
f.write(str(metrics))
f.close()

In [ ]:
rf_cm = confusion_matrix(y_test, y_pred_rf)
rf_disp = ConfusionMatrixDisplay(confusion_matrix=rf_cm)
rf_disp.plot()
plt.show()

dm_cm = confusion_matrix(y_test, y_pred_dm)
dm_disp = ConfusionMatrixDisplay(confusion_matrix=dm_cm)
dm_disp.plot()
plt.show()